In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import db_queries as db
from get_draws.api import get_draws

!whoami
!date

ndbs
Mon Oct 12 17:13:03 PDT 2020


In [2]:
!pwd

/ihme/homes/ndbs/vivarium_data_analysis/pre_processing/lsff_project


In [3]:
import sys, os.path

sys.path.append(os.path.abspath("../..")) # Adds the vivarium_data_analysis directory to the python path
from pre_processing import id_helper as idh

In [11]:
%load_ext autoreload
%autoreload 2

# Find locations - I want all countries (195?)

Looks like I'll have to settle for filtering to `location_type == 'admin0'` for now, since I don't know how to get exactly the right id's.

In [5]:
all_locations = db.get_ids('location')
all_locations.head()

,location_id,location_name,location_type,location_description
0,1,Global,global,None
1,6,China,admin0,admin0
2,7,Democratic People's Republic of Korea,admin0,admin0
3,8,Taiwan (Province of China),admin0,admin0
4,10,Cambodia,admin0,admin0


In [7]:
all_locations.shape

(34133, 4)

In [9]:
admin0_locations = all_locations.query('location_type=="admin0"')
admin0_locations.shape

(299, 4)

In [12]:
admin0_locations.head(10)

,location_id,location_name,location_type,location_description
1,6,China,admin0,admin0
2,7,Democratic People's Republic of Korea,admin0,admin0
3,8,Taiwan (Province of China),admin0,admin0
4,10,Cambodia,admin0,admin0
5,11,Indonesia,admin0,admin0
6,12,Lao People's Democratic Republic,admin0,admin0
7,13,Malaysia,admin0,admin0
8,14,Maldives,admin0,admin0
9,15,Myanmar,admin0,admin0
10,16,Philippines,admin0,admin0


In [17]:
idh.get_entity_from_table(admin0_locations)

'location'

In [24]:
idh.search_id_table(admin0_locations, 'China')

,location_id,location_name,location_type,location_description
1,6,China,admin0,admin0
3,8,Taiwan (Province of China),admin0,admin0
299,60356,China (WPRO),admin0,"China as defined by the WHO. Contains Taiwan, ..."


In [19]:
admin0_locations.query('location_description != "admin0"')

,location_id,location_name,location_type,location_description
25,34,Azerbaijan,admin0,INF-5395 - Subnationals for Azerbaijan (locati...
29,38,Mongolia,admin0,INF-5395 Mongolia subnationals
58,72,New Zealand,admin0,INF-5319
65,80,France,admin0,INF-5395 France subnationals
71,86,Italy,admin0,Added for INF-5280
78,93,Sweden,admin0,LBD added via INF-5319
89,108,Belize,admin0,INF-5395 Belize subnationals.
106,127,El Salvador,admin0,INF-5395 El-Salvador subnationals
227,349,Greenland,admin0,LBD added via INF-5319
237,371,Netherlands Antilles (including Aruba),admin0,nonsovereign


In [20]:
# Looks like the Cyprus we want has admin0 for the location_description
idh.search_id_table(admin0_locations, 'Cyprus')

,location_id,location_name,location_type,location_description
62,77,Cyprus,admin0,admin0
287,53483,Turkish Republic of Northern Cyprus,admin0,Added per INF-4646
288,53484,Cyprus (not including TRNC),admin0,Added per INF-4646


In [21]:
# But there's only one Mongolia, and it *doesn't* have admin0 for the location_description...ugh!
idh.search_id_table(admin0_locations, 'Mongolia')

,location_id,location_name,location_type,location_description
29,38,Mongolia,admin0,INF-5395 Mongolia subnationals


# Call `get_draws` to pull stunting prevalence for all 'admin0' locations

In [34]:
# Check that my ids_in() function works
idh.ids_in(admin0_locations)[:20]

[6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27]

In [35]:
# Get the id for the 'Under 5' age group
idh.list_ids('age_group', 'Under 5')

1

In [25]:
# Show get_draws() docstring for easy reference, so I don't catastrophically forget a filtering parameter 
help(get_draws)

Help on function get_draws in module get_draws.api:

get_draws(gbd_id_type, gbd_id, source, measure_id=None, location_id=None, year_id=None, age_group_id=None, sex_id=None, metric_id=None, version_id=None, status='best', gbd_round_id=7, num_workers=None, downsample=False, decomp_step=None, n_draws=None)
    Get draws for given arguments.
    
    Args:
        gbd_id_type(str or strlist): The gbd entity to retrieve draws for.
            Currently only 'cause_id', 'modelable_entity_id', 'rei_id',
            'sequela_id', and 'indicator_component_id are supported.
    
            Note -- gbd_id_type arguments must be paired 1 to 1 with
            gbd_id arguments if you are specifying multiple gbd_id_types.
            For example::
    
                get_draws(gbd_id_type=['cause_id', 'rei_id', rei_id],
                        gbd_id=[294, 82, 83], ... )
    
        gbd_id(int or intlist): The id(s) within the gbd_id_type(s) whose
            draws will be retrieved.
            

In [36]:
stunting = get_draws(
    'rei_id',
    gbd_id=idh.list_ids('rei', 'Child stunting'),
    source='exposure',
    measure_id=idh.list_ids('measure', 'prevalence'),
    location_id=idh.ids_in(admin0_locations),
    year_id=2019,
    age_group_id=idh.list_ids('age_group', 'Under 5'),
    sex_id=idh.list_ids('sex', 'Both'),
    gbd_round_id=idh.list_ids('gbd_round', '2019'),
    status='best',
    decomp_step='step4',
)
stunting.head()

/ihme/code/central_comp/miniconda3_10_30_2019/envs/v95/lib/python3.7/site-packages/get_draws/transforms/automagic.py:169: UserWarning: Final df is missing location_id 188,296,297,301,302,303,304,306,307,308,309,310,311,312,314,315,316,317,321,322,323,324,326,327,328,329,330,333,334,335,336,337,340,341,342,343,344,346,347,348,353,357,358,359,360,362,365,371,373,377,378,379,381,383,384,386,388,389,390,397,398,399,400,402,403,404,405,406,407,408,409,410,412,414,417,418,419,420,425,426,427,428,429,430,431,432,4627,4628,4706,4707,4708,53483,53484,53485,53495,53496,53498,53499,53501,53502,53509,53510,53511,60356 that was requested. Is this expected? Maybe age/sex restriction?
  str(v) for v in filter_dict[k] if v not


,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,241,8951.0,6,2019,1,3,cat3,5,3,0.303820,...,0.320197,0.322260,0.296655,0.311466,0.282390,0.319573,0.306733,0.304351,0.308995,0.343090
1,241,8951.0,7,2019,1,3,cat3,5,3,0.332241,...,0.308521,0.365996,0.366902,0.324369,0.349718,0.372429,0.328082,0.314922,0.320968,0.357297
2,241,8951.0,8,2019,1,3,cat3,5,3,0.188571,...,0.166204,0.174744,0.173367,0.172879,0.179043,0.164431,0.172707,0.169683,0.145522,0.169720
3,241,8951.0,10,2019,1,3,cat3,5,3,0.345206,...,0.327548,0.355528,0.346804,0.347003,0.352576,0.342982,0.351583,0.343957,0.353110,0.358065
4,241,8951.0,11,2019,1,3,cat3,5,3,0.274161,...,0.274958,0.272785,0.274118,0.275281,0.269320,0.272719,0.267390,0.272035,0.267614,0.269422


### Woo hoo, it looks like the id's I got results for are probably the 195 countries I want

Oh, I wonder if I could have used a location set to figure this out...

In [37]:
# See how many id's were in the warning
len([188,296,297,301,302,303,304,306,307,308,309,310,311,312,314,315,316,317,321,322,323,324,326,327,328,329,330,333,334,335,336,337,340,341,342,343,344,346,347,348,353,357,358,359,360,362,365,371,373,377,378,379,381,383,384,386,388,389,390,397,398,399,400,402,403,404,405,406,407,408,409,410,412,414,417,418,419,420,425,426,427,428,429,430,431,432,4627,4628,4706,4707,4708,53483,53484,53485,53495,53496,53498,53499,53501,53502,53509,53510,53511,60356])

104

In [38]:
# see how many id's weren't in the warning - woo hoo, 195 countries!
299-104

195

In [49]:
# 195 countries
stunting.location_id.nunique()

195

# Look up `modelable_entity_id`'s to see what data I got

In [42]:
idh.ids_to_names('modelable_entity', 8951)

modelable_entity_id
8951    Stunting Between -2 SD and -1 SD (post-ensemble)
Name: modelable_entity_name, dtype: object

In [44]:
stunting.modelable_entity_id.unique()

array([8951., 8949., 8950.,   nan])

In [45]:
idh.ids_to_names('modelable_entity', 8951, 8949, 8950)

modelable_entity_id
8949            Severe Stunting, < -3 SD (post-ensemble)
8950    Stunting Between -3 SD and -2 SD (post-ensemble)
8951    Stunting Between -2 SD and -1 SD (post-ensemble)
Name: modelable_entity_name, dtype: object

In [47]:
# See what's in the NaN rows... it's 'cat4'
# Oh, this must be the non-stunted category
stunting.query('modelable_entity_id != modelable_entity_id').head()

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
585,241,NaN,6,2019,1,3,cat4,5,3,0.586469,...,0.559981,0.559425,0.591711,0.569846,0.608266,0.570140,0.581383,0.573116,0.584947,0.547516
586,241,NaN,7,2019,1,3,cat4,5,3,0.376614,...,0.399348,0.348976,0.329044,0.386833,0.358249,0.329584,0.375214,0.377054,0.372785,0.326234
587,241,NaN,8,2019,1,3,cat4,5,3,0.783036,...,0.804158,0.793913,0.795331,0.798752,0.793988,0.806292,0.795829,0.799226,0.830730,0.802726
588,241,NaN,10,2019,1,3,cat4,5,3,0.240645,...,0.233571,0.228693,0.247621,0.223764,0.222051,0.246651,0.257327,0.262290,0.271103,0.234738
589,241,NaN,11,2019,1,3,cat4,5,3,0.382266,...,0.384585,0.385113,0.383372,0.380257,0.387502,0.385385,0.390941,0.385616,0.389558,0.385415


# Process the stunting data

In [39]:
stunting.shape

(780, 1009)

In [46]:
# 780 rows = 195 countries x 4 stunting categories
780/195

4.0

In [52]:
draw_cols = [c for c in stunting.columns if 'draw' in c]
len(draw_cols)

1000

In [57]:
index_cols = stunting.columns.difference(draw_cols)
index_cols

Index(['age_group_id', 'location_id', 'measure_id', 'metric_id',
       'modelable_entity_id', 'parameter', 'rei_id', 'sex_id', 'year_id'],
      dtype='object')

In [113]:
# Standard definition of stunting is Z < -2, so omit cats 3 and 4
df = stunting.query("modelable_entity_id in [8949, 8950]")
# Now sum the prevalences over cats 1 and 2 to get total stunting prevalence
df = df.groupby(index_cols.difference(['modelable_entity_id','parameter']).to_list()).sum()
df.head()

modelable_entity_id  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id                        
1            6           5          3         241    3      2019                 17899.0   
             7           5          3         241    3      2019                 17899.0   
             8           5          3         241    3      2019                 17899.0   
             10          5          3         241    3      2019                 17899.0   
             11          5          3         241    3      2019                 17899.0   

                                                                       draw_0  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id             
1            6           5          3         241    3      2019     0.109711   
             7           5          3         241    3      2019     0.291144   
             8           5          3         241    3      2019     0.028393   
             10          5          3         241    3      2019     0.414149   
             11          5          3         241    3      2019     0.343574   

                                                                       draw_1  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id             
1            6           5          3         241    3      2019     0.110568   
             7           5          3         241    3      2019     0.293317   
             8           5          3         241    3      2019     0.030138   
             10          5          3         241    3      2019     0.402156   
             11          5          3         241    3      2019     0.339752   

                                                                      draw_10  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id             
1            6           5          3         241    3      2019     0.107988   
             7           5          3         241    3      2019     0.296949   
             8           5          3         241    3      2019     0.029102   
             10          5          3         241    3      2019     0.415457   
             11          5          3         241    3      2019     0.344543   

                                                                     draw_100  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id             
1            6           5          3         241    3      2019     0.110397   
             7           5          3         241    3      2019     0.307726   
             8           5          3         241    3      2019     0.028720   
             10          5          3         241    3      2019     0.384014   
             11          5          3         241    3      2019     0.341906   

                                                                     draw_101  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id             
1            6           5          3         241    3      2019     0.113155   
             7           5          3         241    3      2019     0.286081   
             8           5          3         241    3      2019     0.029314   
             10          5          3         241    3      2019     0.393118   
             11          5          3         241    3      2019     0.337096   

                                                                     draw_102  \
age_group_id location_id measure_id metric_id rei_id sex_id year_id             
1            6           5          3         241    3      2019     0.119084   
             7           5          3         241    3      2019     0.302242   
             8           5          3         241    3      2019     0.029320   
             10          5          3         241    3      2019     0.383110   
             11          5          3         241    3      2019     0.337942   

                

In [114]:
# Make sure we have one row per country
df.shape

(195, 1001)

In [115]:
# Take the mean over draws to get a single stunting prevalence estimate for each country
df = df.drop(columns='modelable_entity_id')
df = df.mean(axis=1).reset_index().rename(columns={0:'stunting_prevalence'})
df.head()

,age_group_id,location_id,measure_id,metric_id,rei_id,sex_id,year_id,stunting_prevalence
0,1,6,5,3,241,3,2019,0.110515
1,1,7,5,3,241,3,2019,0.297625
2,1,8,5,3,241,3,2019,0.029480
3,1,10,5,3,241,3,2019,0.403864
4,1,11,5,3,241,3,2019,0.341780


# Add location names to stunting dataframe

In [116]:
locations = idh.ids_to_names('location', *df.location_id.unique()).reset_index()
locations.head()

,location_id,location_name
0,6,China
1,7,Democratic People's Republic of Korea
2,8,Taiwan (Province of China)
3,10,Cambodia
4,11,Indonesia


In [117]:
df = df.merge(locations, on='location_id')
df.head()

,age_group_id,location_id,measure_id,metric_id,rei_id,sex_id,year_id,stunting_prevalence,location_name
0,1,6,5,3,241,3,2019,0.110515,China
1,1,7,5,3,241,3,2019,0.297625,Democratic People's Republic of Korea
2,1,8,5,3,241,3,2019,0.029480,Taiwan (Province of China)
3,1,10,5,3,241,3,2019,0.403864,Cambodia
4,1,11,5,3,241,3,2019,0.341780,Indonesia


# Add population data to dataframe

In [118]:
population = db.get_population(
    age_group_id=idh.find_ids('age_group', 'under', case=False),
    location_id=idh.ids_in(locations),
    year_id=2019,
    gbd_round_id=idh.list_ids('gbd_round', '2019'),
    decomp_step='step4',
    with_ui=False,
)

print(population.shape)
population.head()

(195, 6)


,age_group_id,location_id,year_id,sex_id,population,run_id
0,1,6,2019,3,8.149086e+07,192
1,1,7,2019,3,1.665364e+06,192
2,1,8,2019,3,9.825697e+05,192
3,1,10,2019,3,1.767535e+06,192
4,1,11,2019,3,1.970635e+07,192


In [119]:
df = df.merge(population, on=['age_group_id', 'location_id', 'year_id', 'sex_id'])
df.head()

,age_group_id,location_id,measure_id,metric_id,rei_id,sex_id,year_id,stunting_prevalence,location_name,population,run_id
0,1,6,5,3,241,3,2019,0.110515,China,8.149086e+07,192
1,1,7,5,3,241,3,2019,0.297625,Democratic People's Republic of Korea,1.665364e+06,192
2,1,8,5,3,241,3,2019,0.029480,Taiwan (Province of China),9.825697e+05,192
3,1,10,5,3,241,3,2019,0.403864,Cambodia,1.767535e+06,192
4,1,11,5,3,241,3,2019,0.341780,Indonesia,1.970635e+07,192


# Calculate number stunted and sort descending

In [120]:
df['number_stunted'] = df['stunting_prevalence']*df['population']
df.head()

,age_group_id,location_id,measure_id,metric_id,rei_id,sex_id,year_id,stunting_prevalence,location_name,population,run_id,number_stunted
0,1,6,5,3,241,3,2019,0.110515,China,8.149086e+07,192,9.005922e+06
1,1,7,5,3,241,3,2019,0.297625,Democratic People's Republic of Korea,1.665364e+06,192,4.956548e+05
2,1,8,5,3,241,3,2019,0.029480,Taiwan (Province of China),9.825697e+05,192,2.896641e+04
3,1,10,5,3,241,3,2019,0.403864,Cambodia,1.767535e+06,192,7.138437e+05
4,1,11,5,3,241,3,2019,0.341780,Indonesia,1.970635e+07,192,6.735246e+06


In [121]:
df.sort_values('number_stunted', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.index = df.index+1
df.head()

,age_group_id,location_id,measure_id,metric_id,rei_id,sex_id,year_id,stunting_prevalence,location_name,population,run_id,number_stunted
1,1,163,5,3,241,3,2019,0.398865,India,1.170708e+08,192,4.669548e+07
2,1,165,5,3,241,3,2019,0.419606,Pakistan,3.052648e+07,192,1.280908e+07
3,1,214,5,3,241,3,2019,0.365739,Nigeria,3.352164e+07,192,1.226015e+07
4,1,6,5,3,241,3,2019,0.110515,China,8.149086e+07,192,9.005922e+06
5,1,179,5,3,241,3,2019,0.404214,Ethiopia,1.667967e+07,192,6.742155e+06
6,1,11,5,3,241,3,2019,0.341780,Indonesia,1.970635e+07,192,6.735246e+06
7,1,161,5,3,241,3,2019,0.398797,Bangladesh,1.374591e+07,192,5.481824e+06
8,1,171,5,3,241,3,2019,0.376227,Democratic Republic of the Congo,1.362947e+07,192,5.127766e+06
9,1,16,5,3,241,3,2019,0.300919,Philippines,1.265431e+07,192,3.807918e+06
10,1,189,5,3,241,3,2019,0.394217,United Republic of Tanzania,9.307277e+06,192,3.669087e+06


# Calculate total number stunted globally

In [138]:
global_stunted_population = df['number_stunted'].sum().round()
global_stunted_population

176909741.0

# Make nicer looking dataframes and save them

## First, all countries

In [161]:
all_countries_ranked = df[['location_name', 'location_id', 'stunting_prevalence', 'population', 'number_stunted']]
all_countries_ranked.rename(columns={'population': 'population_under_5'}, inplace=True)
all_countries_ranked.head()

,location_name,location_id,stunting_prevalence,population_under_5,number_stunted
1,India,163,0.398865,1.170708e+08,4.669548e+07
2,Pakistan,165,0.419606,3.052648e+07,1.280908e+07
3,Nigeria,214,0.365739,3.352164e+07,1.226015e+07
4,China,6,0.110515,8.149086e+07,9.005922e+06
5,Ethiopia,179,0.404214,1.667967e+07,6.742155e+06


In [162]:
all_countries_ranked['cumulative_number_stunted'] = all_countries_ranked['number_stunted'].cumsum()
all_countries_ranked.head()

,location_name,location_id,stunting_prevalence,population_under_5,number_stunted,cumulative_number_stunted
1,India,163,0.398865,1.170708e+08,4.669548e+07,4.669548e+07
2,Pakistan,165,0.419606,3.052648e+07,1.280908e+07,5.950456e+07
3,Nigeria,214,0.365739,3.352164e+07,1.226015e+07,7.176472e+07
4,China,6,0.110515,8.149086e+07,9.005922e+06,8.077064e+07
5,Ethiopia,179,0.404214,1.667967e+07,6.742155e+06,8.751279e+07


In [163]:
all_countries_ranked['cum_percent_global_stunted_pop'] = (
    all_countries_ranked['cumulative_number_stunted']/global_stunted_population
)
all_countries_ranked.head(35)

,location_name,location_id,stunting_prevalence,population_under_5,number_stunted,cumulative_number_stunted,cum_percent_global_stunted_pop
1,India,163,0.398865,1.170708e+08,4.669548e+07,4.669548e+07,0.263951
2,Pakistan,165,0.419606,3.052648e+07,1.280908e+07,5.950456e+07,0.336355
3,Nigeria,214,0.365739,3.352164e+07,1.226015e+07,7.176472e+07,0.405657
4,China,6,0.110515,8.149086e+07,9.005922e+06,8.077064e+07,0.456564
5,Ethiopia,179,0.404214,1.667967e+07,6.742155e+06,8.751279e+07,0.494675
6,Indonesia,11,0.341780,1.970635e+07,6.735246e+06,9.424804e+07,0.532746
7,Bangladesh,161,0.398797,1.374591e+07,5.481824e+06,9.972986e+07,0.563733
8,Democratic Republic of the Congo,171,0.376227,1.362947e+07,5.127766e+06,1.048576e+08,0.592718
9,Philippines,16,0.300919,1.265431e+07,3.807918e+06,1.086655e+08,0.614243
10,United Republic of Tanzania,189,0.394217,9.307277e+06,3.669087e+06,1.123346e+08,0.634983


In [145]:
all_countries_ranked.shape

(195, 7)

In [146]:
all_countries_ranked.to_csv('countries_ranked_by_number_stunted_2019.csv')

## Now, only countries with stunting prevalence $\ge$ 20%

In [140]:
high_stunting_countries = df.query('stunting_prevalence>=0.2')
high_stunting_countries = high_stunting_countries[
    ['location_name', 'location_id', 'stunting_prevalence', 'population', 'number_stunted']
] 
high_stunting_countries.rename(columns={'population': 'population_under_5'}, inplace=True)
high_stunting_countries.head()

,location_name,location_id,stunting_prevalence,population_under_5,number_stunted
1,India,163,0.398865,1.170708e+08,4.669548e+07
2,Pakistan,165,0.419606,3.052648e+07,1.280908e+07
3,Nigeria,214,0.365739,3.352164e+07,1.226015e+07
5,Ethiopia,179,0.404214,1.667967e+07,6.742155e+06
6,Indonesia,11,0.341780,1.970635e+07,6.735246e+06


In [141]:
high_stunting_countries['cumulative_number_stunted'] = high_stunting_countries['number_stunted'].cumsum()
high_stunting_countries.head()

,location_name,location_id,stunting_prevalence,population_under_5,number_stunted,cumulative_number_stunted
1,India,163,0.398865,1.170708e+08,4.669548e+07,4.669548e+07
2,Pakistan,165,0.419606,3.052648e+07,1.280908e+07,5.950456e+07
3,Nigeria,214,0.365739,3.352164e+07,1.226015e+07,7.176472e+07
5,Ethiopia,179,0.404214,1.667967e+07,6.742155e+06,7.850687e+07
6,Indonesia,11,0.341780,1.970635e+07,6.735246e+06,8.524212e+07


In [143]:
high_stunting_countries['cum_percent_global_stunted_pop'] = (
    high_stunting_countries['cumulative_number_stunted']/global_stunted_population
)
high_stunting_countries.head(35)

,location_name,location_id,stunting_prevalence,population_under_5,number_stunted,cumulative_number_stunted,cum_percent_global_stunted_pop
1,India,163,0.398865,1.170708e+08,4.669548e+07,4.669548e+07,0.263951
2,Pakistan,165,0.419606,3.052648e+07,1.280908e+07,5.950456e+07,0.336355
3,Nigeria,214,0.365739,3.352164e+07,1.226015e+07,7.176472e+07,0.405657
5,Ethiopia,179,0.404214,1.667967e+07,6.742155e+06,7.850687e+07,0.443768
6,Indonesia,11,0.341780,1.970635e+07,6.735246e+06,8.524212e+07,0.481840
7,Bangladesh,161,0.398797,1.374591e+07,5.481824e+06,9.072394e+07,0.512826
8,Democratic Republic of the Congo,171,0.376227,1.362947e+07,5.127766e+06,9.585171e+07,0.541811
9,Philippines,16,0.300919,1.265431e+07,3.807918e+06,9.965963e+07,0.563336
10,United Republic of Tanzania,189,0.394217,9.307277e+06,3.669087e+06,1.033287e+08,0.584076
11,Afghanistan,160,0.504491,6.645141e+06,3.352416e+06,1.066811e+08,0.603026


In [144]:
high_stunting_countries.shape

(74, 7)

In [147]:
high_stunting_countries.to_csv('countries_with_over_20_percent_stunting_ranked_by_number_stunted_2019.csv')